In [3]:
#Importing modules

import os
import numpy as np
import pandas as pd
from tables import *
import h5py
import matplotlib.pyplot as plt
import seaborn as sns


## <div align="center">Start working with FILE data</div>

### The solution is only for s_tract. It should be scaled up to use: 'f_tract' and PTNE Labels

#### Read CSV. Chage .wav to .hdf5

In [4]:
# specify YOUR PATH to the CSV file
csv_path=os.path.join('ESC50_Home','ESC-50-HumanClassification.csv')

# read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# check if all elements in the 'filename' column end with '.wav'
result = (df['filename'].str.endswith('.wav')).all()

# print the result
print(result)

# replace '.wav' with '.hdf5' in the 'filename' column
df['filename'] = df['filename'].str.replace('.wav', '.hdf5')

FileNotFoundError: [Errno 2] No such file or directory: 'ESC50_Home/ESC-50-HumanClassification.csv'

#### Read categories from csv file

In [ ]:
unique_categories = df['category'].unique()

# print the unique categories
print(unique_categories)

count_of_unique_categories = df['category'].nunique()
print(count_of_unique_categories)
print("")
print("Completed")

In [ ]:
# list of categories taken from a blackboard
categories = [
    'Breathing',
    'Coughing',
    'Footsteps',
    'Laughing',
    'Sneezing',
    'Snoring',
    'Toilet_flush',
    'Vacuum_cleaner',
    'Washing_machine',
]

# change the capital letter to lowercase
categories = [category.lower() for category in categories]

# print the modified list
print(categories)
print("")
print("Completed")

%timeit

#### Make a dictionary with keys=categories and values=category_element

In [ ]:
# Create empty dict for storing filenames for each category
filenames_by_category  = {}
number_of_files = 0

# iterate over the categories
for category in categories:
    # get all the filenames for the current category
    filenames = df.loc[df['category'].isin([category]), 'filename'].values.tolist()
    number_of_files += len(filenames)
    # create a key-value pair in the dictionary with the current category as the key and the list of filenames as the value
    filenames_by_category[category] = filenames

print("The number of files %f" %number_of_files)
print("Completed")

#### Make a dictionary with keys=categories and values=path_to_element

In [ ]:
# Create empty dict for storing paths to files for each category
hdf5_paths_per_category = {}

# In order to control the flow of the code and not miss a change I keep trck on the number of files.
counter_files = 0
counter_categories = 0 


# iterate over the categories
for category in categories:
    counter_categories += 1
    
    # Create an empty list where I store paths for one type of category
    all_paths_for_same_category = list()
    
    # iterate over the list that belongs to exact category
    for i in filenames_by_category[category]:
        counter_files += 1
        
        # Create variable that store a single path
        # CHANGE TO YOUR PATH
        temp_hdf5_path=os.path.join('ESC50_Home', 'tracts', i)
        
        # Store paths for one type of category in a list.
        # With the new category the list will be updated to 0
        all_paths_for_same_category.append(temp_hdf5_path)
        
    # Store a list according to the category
    hdf5_paths_per_category[category] = all_paths_for_same_category


print(type(hdf5_paths_per_category["snoring"]))
print(len(hdf5_paths_per_category["snoring"]))

print("The number of categories %f" %counter_categories)
print("The number of files %f" %counter_files)
print("Completed")

## <div align="center">Prepare functions</div>

### Functions for features

In [ ]:
def create_arr_all_files_of_category(category_name):
    """ This code extracts data from hdf5 files, access the s_tract dataset in each file,  
    and append a portion of that data to a list called list_of_data"""
    
    counter_files = 0
    list_of_data = list() # Create list. Here we will store all arrays for 1 category
    
    for i in hdf5_paths_per_category[category_name]:
        
        # Here we iterate over every file of one category
        with h5py.File(i, 'r') as f:
            
            # access the "s_tract" dataset in the file
            # s_tract has shape (100, 8497), class 'numpy.ndarray
            s_tract = f['s_tract'][()]
            # Append a file and get rid of zeros in the bottom and in the top
            list_of_data.append(s_tract[6:-6,:])
        
        counter_files += 1
        
    print("Completed")
    return list_of_data

## Here we extract features from a histogram

In [ ]:
def hist_and_bins(array):
    """Should receive flat array.
    The input is an array with a shape of (88, 8497) or (88, 8498).
    The output will be features. The number of features depends on how many bins you define"""
    
    # I keep feature consistency
#     arr = array
    arr = array[array <= 200] # Here I delete all the value in array greater than 90.
    # I do it because after I check the original histogram(the graph) based on visual observation
    # I decided to cut off values greater than 
    
    # You can decide by yourself if you want to cut of. If you decide to cut of then you need to rearrange bins
    # That are in the following line
    
    bins = [5,50,90] # 2 features --> You can change to 2 features, go to SVM part uncoment Plotting part and see the plot. 
# the plot works only with 2 features.
    bins = [0,25,50,100] # 2
#     bins = [0, 22.5, 45, 67.5, 90] # 4 features
#     bins = [0, 20, 40, 60, 80, 100, 120] # 5 features
    
    # Compute histogram
    # We extract bin, but we do not use it in the future.
    features, bins = np.histogram(arr, bins=bins) 
    
    # Extract frequency from each bin
#     print("Bin Edges: ", bins)
#     print("Frequency: ", features)
    return features

In [ ]:
def flatten_and_get_features_from_hist(arr):
    # Part of the hist_and_bin function
    feature_new = list()
    
    # Array is a list of 40 elements
    for i in arr:
        # i is np array with a shape of (88, 8498)
        
        k = i.flatten() # make np array flat
        hist_feature = hist_and_bins(k) # get features from a histogram
        
        feature_new.append(hist_feature) # store in the list
        
    print("Completed flatten_and_get_features_from_hist")   
    return feature_new

In [ ]:
def split_arrat_func(file_2D_data, sep_number):
    
    # Preparation before splitting
    length_of_data = len(file_2D_data)
    sub_arr_size = length_of_data // sep_number
    safe_separate = sub_arr_size * sep_number
    
    # The main part - splitting
    splited_lists = np.split(np.array(file_2D_data[:safe_separate]), sep_number)
    
#     print("Completed - split_arrat_func()")
#     print("")
    return splited_lists
    

In [ ]:
def get_feature(splited_array):
    list_of_features = np.array([]) # 1
#     list_of_features = list() # 2
    # Features consistency cut out the border.
    
    for i in splited_array:
        arr_one_flat = i.flatten() # 246413
        hist1, bins1 = np.histogram(arr_one_flat,bins=10)
#         list_of_features.append(hist1)
        list_of_features = np.append(list_of_features, hist1) # 1

    
#     list_of_features = np.array(list_of_features) # 2
    
    return list_of_features

###  Let's wrap everything together into the prep to SVM function

In [1]:
def prepare_data_for_SVM(array_with_catigories):
# def prepare_data_for_SVM(array_with_catigories, num_of_features):
    
    label_array = list() # For collecting labels
    num_label = 0 # Every iteration of a category num_label will +1 and label_array will recieve new values
    
    all_categories_features = list() # Global array for features
    
    for i in array_with_catigories:
        print("")
        print("Category : ", i)
        one_category_features = list() # Local array for features only for one category 

        arr_all_files_one_category = create_arr_all_files_of_category(i) # List, all of 40 files of ine category. One array has shape ((88, 8498))
        
        # Get access to individual file (88, 8498)
        for i in arr_all_files_one_category:
            
            splited_lists = split_arrat_func(i, 3) # list (3). Every element has a shape (27,8498)   
            hey = get_feature(splited_lists) #  np.array - get features for one file - for example (30,)
            one_category_features.append(hey) # list - store features in the list
            
        one_category_features = np.array(one_category_features) # (40, 30) - feature set(30) for 40 samples - one category
        all_categories_features.append(one_category_features) # store seatures set in a list. Len will be 9.

        label_1 = np.zeros(len(arr_all_files_one_category), dtype=int) + num_label

        label_array.append(label_1) 
        
        num_label += 1
        
    print("Saving...")

### Create features and labels

In [ ]:
features_array,label_array = prepare_data_for_SVM(categories) # "FULLTEST"

In [ ]:
# Check the shapes if they're ready to feed SVM

print(features_array.shape)
print(label_array.shape)
print(type(features_array))
print(type(label_array))
print("Cool")

### Test SVM

In [ ]:
# Sklearn modules & classes
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn import metrics

from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score



X = features_array
y = label_array
print(X.shape)
print(y.shape)

# Create training and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)


sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# 'rbf', 'poly', 'sigmoid', 'linear'
# degree=3
# Instantiate the Support Vector Classifier (SVC)
svc = SVC(C=1.0, random_state=10, kernel='poly')
 
# Fit the model
svc.fit(X_train_std, y_train)


# Make the predictions
y_predict = svc.predict(X_test_std)

###################################### PLOTTING #############################
# # Visualize the decision regions
# plt.figure(figsize=(12,6))
# plot_decision_regions(X_train_std, y_train, clf=svc, legend=2)

# # Add the support vectors
# plt.scatter(svc.support_vectors_[:, 0],
#             svc.support_vectors_[:, 1],
#             s=700, facecolors='none')

# # Add the axis labels
# plt.xlabel('x1')
# plt.ylabel('x2')

# # Show the plot
# plt.show()
############################################################################
acc_score = accuracy_score(y_test, y_predict)

cm = confusion_matrix(y_test, y_predict)

# Extract the values from the confusion matrix
TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]

# Calculate the accuracy of the model
accuracy = (TP + TN) / (TP + TN + FP + FN)

print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)
print("Accuracy:", accuracy)

print("")
print("")
cr = classification_report(y_test, y_predict)
print(cr)

print("")
# Plot the confusion matrix using seaborn
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()